In [0]:
import math
import numpy as np
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    FloatType,
    IntegerType,
    ArrayType,
    MapType,
    DateType,
    DoubleType,
    BooleanType,
    LongType,
)

In [0]:
df = spark.sql("""

  SELECT unioned.*,`dim`.`Hierarchy_Description`,dim.MEDIAN_AVERAGE 	 ,dim.ADD_TO_PROFILE 	,dim.LEAF_NODE_FLAG	,dim.TOTAL_FLAG, dim.YEAR as WHOLE_YEAR

  FROM (select * FROM environics.demostats_2023_all_pc_vars
union
select * from environics.demostats_2026_all_pc_vars
union
select * from environics.demostats_2028_all_pc_vars
union
select * from environics.demostats_2033_all_pc_vars
union
select * from environics.newtocanada_2024q1p1_all_pc_vars
union 
select * from environics.newtocanada_2024q1p2_all_pc_vars
)
  As unioned
left join hyper.master_demostats as dim ON  substr(dim.variable, 4) = substr(unioned.VARIABLES, 4)

  WHERE unioned.CATEGORY IN (
      SELECT DISTINCT dim.Category 
      FROM hyper.master_demostats_1001  AS dim 
      WHERE dim.ADD_TO_PROFILE = 1 or dim.YEAR IS NOT NULL
  )
AND dim.ADD_TO_PROFILE = 1 or dim.YEAR IS NOT NULL
""")

display(df)

In [0]:
df= df.withColumn('YEAR', when(substring('VARIABLES', 0, 3)=='EHY', 2018).when(substring('VARIABLES', 0, 3)=='ECY', 2023).when(substring('VARIABLES', 0, 3)=='P3Y', 2026).when(substring('VARIABLES', 0, 3)=='P5Y', 2028).when(substring('VARIABLES', 0, 3)=='P0Y', 2033))

df = df.withColumnRenamed('VARIABLES', 'variable')
df = df.withColumnRenamed('VALUES', 'value')

# display(df)

In [0]:
# remove all the special characters in Description column
cleaned_df = (
    df.withColumn(
        "Description", regexp_replace("Description", " \\(Current Year \\$\\)", "")
    )
    .withColumn(
        "Description", regexp_replace("Description", "Côte d'Ivoire", "Cote d Ivoire")
    )
    .withColumn(
        "Description", regexp_replace("Description", r"[$,.\-" + "'" + r"()]", "")
    )
    )
cleaned_df_2024 = cleaned_df.filter((col('YEAR') == '2024') | (col('WHOLE_YEAR').isNotNull()))

In [0]:
display(cleaned_df_2024.filter(col('Category')=='23Q2 to 24Q1 - Temporary Resident - Region of Citizenship'))

In [0]:

# cleaned_df_2024 = cleaned_df.filter((col('YEAR') == '2024') | (col('WHOLE_YEAR').isNotNull()))
# pivot_df_2024 = (
#     cleaned_df_2024.groupBy("CODE")
#     .pivot("variable")
#     .agg(first("value"))
#     .withColumnRenamed("CODE","POSTALCODE")
# )

cleaned_df_2023 = cleaned_df.filter((col('YEAR').isin(2023, 2024)) | (col('WHOLE_YEAR').isNotNull()))
pivot_df_2023 = (
    cleaned_df_2023.groupBy("CODE")
    .pivot("variable")
    .agg(first("value"))
    .withColumnRenamed("CODE","POSTALCODE")
)
cleaned_df_2026 = cleaned_df.filter(col('YEAR')=='2026')
pivot_df_2026 = (
    cleaned_df_2026.filter(col('YEAR')=='2026').groupBy("CODE")
    .pivot("variable")
    .agg(first("value"))
    .withColumnRenamed("CODE","POSTALCODE")
)
cleaned_df_2028 = cleaned_df.filter(col('YEAR')=='2028')
pivot_df_2028 = (
    cleaned_df_2028.filter(col('YEAR')=='2028').groupBy("CODE")
    .pivot("variable")
    .agg(first("value"))
    .withColumnRenamed("CODE","POSTALCODE")
)
cleaned_df_2033 = cleaned_df.filter(col('YEAR')=='2033')
pivot_df_2033 = (
    cleaned_df_2033.filter(col('YEAR')=='2033').groupBy("CODE")
    .pivot("variable")
    .agg(first("value"))
    .withColumnRenamed("CODE","POSTALCODE")
)

# cleaned_df_immi = cleaned_df.filter(col('WHOLE_YEAR').isNotNull())
# pivot_df_immi = (
#     cleaned_df_immi.groupBy("CODE")
#     .pivot("variable")
#     .agg(first("value"))
#     .withColumnRenamed("CODE","POSTALCODE")
# )

In [0]:
display(cleaned_df_2023.filter(col('WHOLE_YEAR')==2024))

In [0]:


# collist2024 = list(set(pivot_df_2024.columns) - set(['POSTALCODE', 'Total Household Population', 'Total Household Population 15 Years Or Over', 'Total Households', 'Total Population']))
collist2023 = list(set(pivot_df_2023.columns) - set(['POSTALCODE', 'Total Household Population', 'Total Household Population 15 Years Or Over', 'Total Households', 'Total Population']))
collist2026 = list(set(pivot_df_2026.columns) - set(['POSTALCODE', 'Total Household Population', 'Total Household Population 15 Years Or Over', 'Total Households', 'Total Population']))
collist2028 = list(set(pivot_df_2028.columns) - set(['POSTALCODE', 'Total Household Population', 'Total Household Population 15 Years Or Over', 'Total Households', 'Total Population']))
collist2033 = list(set(pivot_df_2033.columns) - set(['POSTALCODE', 'Total Household Population', 'Total Household Population 15 Years Or Over', 'Total Households', 'Total Population']))

# collist_immi = list(set(pivot_df_immi.columns) - set(['POSTALCODE', 'Total Household Population', 'Total Household Population 15 Years Or Over', 'Total Households', 'Total Population']))

In [0]:

# melted_df_2024 = pivot_df_2024.melt(
#     ids=['POSTALCODE'], values=collist2024,
#     variableColumnName="variable", valueColumnName="val"
# ).join(cleaned_df_2024.select('variable','Description', 'Category','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').distinct(), on='variable', how='left').join(pivot_df_2024.select('POSTALCODE', col('ECYBASHHD').alias('TOTAL_HOUSEHOLDS'), col('ECYBAS15HP').alias('TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER'), col('ECYBASHPOP').alias('TOTAL_HOUSEHOLD_POPULATION'), col('ECYBASPOP').alias('TOTAL_POPULATION')), on='POSTALCODE', how='inner').select('variable','POSTALCODE', 'Description', 'Category', 'val', 'TOTAL_HOUSEHOLDS', 'TOTAL_HOUSEHOLD_POPULATION', 'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER', 'TOTAL_POPULATION','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').withColumn('YEAR', lit(2024))


melted_df_2023 = pivot_df_2023.melt(
    ids=['POSTALCODE'], values=collist2023,
    variableColumnName="variable", valueColumnName="val"
).join(cleaned_df_2023.select('variable','Description', 'Category','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').distinct(), on='variable', how='left').join(pivot_df_2023.select('POSTALCODE', col('ECYBASHHD').alias('TOTAL_HOUSEHOLDS'), col('ECYBAS15HP').alias('TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER'), col('ECYBASHPOP').alias('TOTAL_HOUSEHOLD_POPULATION'), col('ECYBASPOP').alias('TOTAL_POPULATION')), on='POSTALCODE', how='inner').select('variable','POSTALCODE', 'Description', 'Category', 'val', 'TOTAL_HOUSEHOLDS', 'TOTAL_HOUSEHOLD_POPULATION', 'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER', 'TOTAL_POPULATION','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').withColumn('YEAR', when(col('variable').startswith('NC24'), lit(2024)).otherwise(lit(2023)))

# melted_df_immi = pivot_df_immi.melt(
#     ids=['POSTALCODE'], values=collist_immi,
#     variableColumnName="variable", valueColumnName="val"
# ).join(cleaned_df_2023.select('variable','Description', 'Category','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').distinct(), on='variable', how='left').join(pivot_df_immi.select('POSTALCODE', col('ECYBASHHD').alias('TOTAL_HOUSEHOLDS'), col('ECYBAS15HP').alias('TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER'), col('ECYBASHPOP').alias('TOTAL_HOUSEHOLD_POPULATION'), col('ECYBASPOP').alias('TOTAL_POPULATION')), on='POSTALCODE', how='inner').select('variable','POSTALCODE', 'Description', 'Category', 'val', 'TOTAL_HOUSEHOLDS', 'TOTAL_HOUSEHOLD_POPULATION', 'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER', 'TOTAL_POPULATION','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').withColumn('YEAR', lit(2023))


melted_df_2026 = pivot_df_2026.melt(
    ids=['POSTALCODE'], values=collist2026,
    variableColumnName="variable", valueColumnName="val"
).join(cleaned_df_2026.select('variable','Description', 'Category','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').distinct(), on='variable', how='left').join(pivot_df_2026.select('POSTALCODE', col('P3YBASHHD').alias('TOTAL_HOUSEHOLDS'), col('P3YBAS15HP').alias('TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER'), col('P3YBASHPOP').alias('TOTAL_HOUSEHOLD_POPULATION'), col('P3YBASPOP').alias('TOTAL_POPULATION')), on='POSTALCODE', how='inner').select('variable','POSTALCODE', 'Description', 'Category', 'val', 'TOTAL_HOUSEHOLDS', 'TOTAL_HOUSEHOLD_POPULATION', 'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER', 'TOTAL_POPULATION','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').withColumn('YEAR', lit(2026))

melted_df_2028 = pivot_df_2028.melt(
    ids=['POSTALCODE'], values=collist2028,
    variableColumnName="variable", valueColumnName="val"
).join(cleaned_df_2028.select('variable','Description', 'Category','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').distinct(), on='variable', how='left').join(pivot_df_2028.select('POSTALCODE', col('P5YBASHHD').alias('TOTAL_HOUSEHOLDS'), col('P5YBAS15HP').alias('TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER'), col('P5YBASHPOP').alias('TOTAL_HOUSEHOLD_POPULATION'), col('P5YBASPOP').alias('TOTAL_POPULATION')), on='POSTALCODE', how='inner').select('variable','POSTALCODE', 'Description', 'Category', 'val', 'TOTAL_HOUSEHOLDS', 'TOTAL_HOUSEHOLD_POPULATION', 'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER', 'TOTAL_POPULATION','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').withColumn('YEAR', lit(2028))

melted_df_2033 = pivot_df_2033.melt(
    ids=['POSTALCODE'], values=collist2033,
    variableColumnName="variable", valueColumnName="val"
).join(cleaned_df_2033.select('variable','Description', 'Category','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').distinct(), on='variable', how='left').join(pivot_df_2033.select('POSTALCODE', col('P0YBASHHD').alias('TOTAL_HOUSEHOLDS'), col('P0YBAS15HP').alias('TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER'), col('P0YBASHPOP').alias('TOTAL_HOUSEHOLD_POPULATION'), col('P0YBASPOP').alias('TOTAL_POPULATION')), on='POSTALCODE', how='inner').select('variable','POSTALCODE', 'Description', 'Category', 'val', 'TOTAL_HOUSEHOLDS', 'TOTAL_HOUSEHOLD_POPULATION', 'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER', 'TOTAL_POPULATION','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG').withColumn('YEAR', lit(2033))

melted_df = melted_df_2023.union(melted_df_2026).union(melted_df_2028).union(melted_df_2033)
# melted_df.show()

In [0]:
display(melted_df.filter(melted_df.Category.contains('23Q2 to 24Q1 - Temporary Resident - Region of Citizenship')))

In [0]:
 
melted_df.write.mode("overwrite").option("overwriteSchema", "True").format("delta").saveAsTable(
    "hyper.ldu_feature_vals_melted_df_master_immi_combined_1015"
)

# **=================================**

In [0]:
%sql
create table hyper.ldu_feature_vals_melted_df_master_immi_combined_0924 as (
select * from hyper.ldu_feature_vals_melted_df_master_variable_0920
union 
select * from hyper.ldu_feature_vals_melted_df_master_immigration_0924

)

In [0]:
df_tmp=spark.sql("""
                 select * from ldu_feature_vals_melted_df_master_immi_combined_1015
                 """)

In [0]:
%sql
SELECT STORE_NUM, BANNER, STORE_PROVINCE, COUNT(DISTINCT STORE_NAME) as count
FROM hyper.store_LDU_attraction_f_halfp_0927
GROUP BY STORE_NUM, BANNER, STORE_PROVINCE
HAVING COUNT(DISTINCT STORE_NAME) > 1

In [0]:
%sql
SELECT STORE.CUSTOMER_GROUP, FIRST(STORE.STORE_NAME), STORE.STORE_NUM, STORE.CHANNEL, STORE.SALES_GROUP,STORE.BANNER,STORE.STORE_STREET, STORE.STORE_PROVINCE,STORE.STORE_LATITUDE,STORE.STORE_LONGITUDE, MELTED.variable, MELTED.Description, LMAP.Language_Grouping, MELTED.Category, MELTED.YEAR,SUM(STORE.PERCENT_ATTRACTION*MELTED.val)  val, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLDS) TOTAL_HOUSEHOLDS, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLD_POPULATION) TOTAL_HOUSEHOLD_POPULATION, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER) TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_POPULATION) TOTAL_POPULATION, first(MELTED.Hierarchy_Description) as Hierarchy_Description, first(MELTED.MEDIAN_AVERAGE) as MEDIAN_AVERAGE, first(MELTED.ADD_TO_PROFILE) as ADD_TO_PROFILE, first(MELTED.LEAF_NODE_FLAG) as LEAF_NODE_FLAG, first(MELTED.TOTAL_FLAG) as TOTAL_FLAG


FROM hyper.store_LDU_attraction_f_halfp_0930 STORE 
LEFT JOIN hyper.ldu_feature_vals_melted_df_master_immi_combined_1001 MELTED ON STORE.POSTALCODE = MELTED.POSTALCODE
LEFT JOIN default.language_mapping LMAP ON MELTED.variable = LMAP.Variable
WHERE MELTED.Category NOT IN ('Trends', 'Attitudes') AND MELTED.val IS NOT NULL
GROUP BY STORE.CUSTOMER_GROUP, STORE.STORE_NAME,STORE.STORE_NUM,  STORE.CHANNEL, STORE.SALES_GROUP, STORE.BANNER,STORE.STORE_STREET,STORE.STORE_LONGITUDE,STORE.STORE_LATITUDE,  STORE.STORE_PROVINCE, MELTED.variable,MELTED.Description,LMAP.Language_Grouping,  MELTED.Category, YEAR

In [0]:
store_profiles_df = spark.sql("""
SELECT STORE.CUSTOMER_GROUP, STORE.STORE_NAME, STORE.STORE_NUM, STORE.CHANNEL, STORE.SALES_GROUP,STORE.BANNER,STORE.STORE_STREET, STORE.STORE_PROVINCE,STORE.STORE_LATITUDE,STORE.STORE_LONGITUDE, MELTED.variable, MELTED.Description, LMAP.Language_Grouping, MELTED.Category, MELTED.YEAR,SUM(STORE.PERCENT_ATTRACTION*MELTED.val)  val, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLDS) TOTAL_HOUSEHOLDS, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLD_POPULATION) TOTAL_HOUSEHOLD_POPULATION, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER) TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_POPULATION) TOTAL_POPULATION, first(MELTED.Hierarchy_Description) as Hierarchy_Description, first(MELTED.MEDIAN_AVERAGE) as MEDIAN_AVERAGE, first(MELTED.ADD_TO_PROFILE) as ADD_TO_PROFILE, first(MELTED.LEAF_NODE_FLAG) as LEAF_NODE_FLAG, first(MELTED.TOTAL_FLAG) as TOTAL_FLAG

-- CASE WHEN MELTED.Category LIKE "Households by%" THEN SUM(STORE.PERCENT_ATTRACTION*MELTED.val)/SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLDS)
--     WHEN MELTED.Category LIKE "Household Population 15 Years or Over%"  THEN SUM(STORE.PERCENT_ATTRACTION*MELTED.val)/SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER)
--     WHEN MELTED.Category LIKE "Household Population%" AND MELTED.Category NOT LIKE "%15 Years or Over%" THEN SUM(STORE.PERCENT_ATTRACTION*MELTED.val)/SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLD_POPULATION)
--     ELSE SUM(STORE.PERCENT_ATTRACTION*MELTED.val)/SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_POPULATION) END STORE_VALUE
FROM hyper.store_LDU_attraction_f_halfp_0927 STORE 
LEFT JOIN hyper.ldu_feature_vals_melted_df_master_immi_combined_0927 MELTED ON STORE.POSTALCODE = MELTED.POSTALCODE
LEFT JOIN default.language_mapping LMAP ON MELTED.variable = LMAP.Variable
WHERE MELTED.Category NOT IN ('Trends', 'Attitudes') AND MELTED.val IS NOT NULL
GROUP BY STORE.CUSTOMER_GROUP, STORE.STORE_NAME,STORE.STORE_NUM,  STORE.CHANNEL, STORE.SALES_GROUP, STORE.BANNER,STORE.STORE_STREET,STORE.STORE_LONGITUDE,STORE.STORE_LATITUDE,  STORE.STORE_PROVINCE, MELTED.variable,MELTED.Description,LMAP.Language_Grouping,  MELTED.Category, YEAR
"""
)


In [0]:
store_profiles_df = spark.sql("""
SELECT STORE.CUSTOMER_GROUP, FIRST(STORE.STORE_NAME) as STORE_NAME, STORE.STORE_NUM, STORE.CHANNEL, STORE.SALES_GROUP,STORE.BANNER,STORE.STORE_STREET, STORE.STORE_PROVINCE,STORE.STORE_LATITUDE,STORE.STORE_LONGITUDE, MELTED.variable, MELTED.Description, LMAP.Language_Grouping, MELTED.Category, MELTED.YEAR,SUM(STORE.PERCENT_ATTRACTION*MELTED.val)  val, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLDS) TOTAL_HOUSEHOLDS, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLD_POPULATION) TOTAL_HOUSEHOLD_POPULATION, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER) TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER, SUM(STORE.PERCENT_ATTRACTION*MELTED.TOTAL_POPULATION) TOTAL_POPULATION, first(MELTED.Hierarchy_Description) as Hierarchy_Description, first(MELTED.MEDIAN_AVERAGE) as MEDIAN_AVERAGE, first(MELTED.ADD_TO_PROFILE) as ADD_TO_PROFILE, first(MELTED.LEAF_NODE_FLAG) as LEAF_NODE_FLAG, first(MELTED.TOTAL_FLAG) as TOTAL_FLAG


FROM hyper.store_LDU_attraction_f_halfp_0927 STORE 
LEFT JOIN hyper.ldu_feature_vals_melted_df_master_immi_combined_1015 MELTED ON STORE.POSTALCODE = MELTED.POSTALCODE
LEFT JOIN default.language_mapping LMAP ON MELTED.variable = LMAP.Variable
WHERE MELTED.Category NOT IN ('Trends', 'Attitudes') AND MELTED.val IS NOT NULL
GROUP BY STORE.CUSTOMER_GROUP, STORE.STORE_NAME,STORE.STORE_NUM,  STORE.CHANNEL, STORE.SALES_GROUP, STORE.BANNER,STORE.STORE_STREET,STORE.STORE_LONGITUDE,STORE.STORE_LATITUDE,  STORE.STORE_PROVINCE, MELTED.variable,MELTED.Description,LMAP.Language_Grouping,  MELTED.Category, YEAR
"""
)


In [0]:

store_profiles_df = store_profiles_df.withColumn(
    "STORE_CHAIN",
    when((col("CUSTOMER_GROUP") == 'AOL_AO LOCAL') | (col("CUSTOMER_GROUP") == 'AON_AO NATL RETAIL'), regexp_replace(col("STORE_NAME"), r"#\d+\s*[A-Z]*\s*[A-Z]*\s*", ""))
    .otherwise(col("CUSTOMER_GROUP"))
)

In [0]:


store_profiles_df = store_profiles_df.withColumn(
    'Region',
    when(col("STORE_PROVINCE") == 'ON', 'ONTARIO')
    .when(col("STORE_PROVINCE") == 'QC', 'QUEBEC')
    .when(col("STORE_PROVINCE").isin(['AB', 'BC', 'MB', 'SK', 'YT', 'NT']), 'WEST')
    .when(col("STORE_PROVINCE").isin(['NS', 'NB', 'NL', 'PE']), 'EAST')
    .otherwise(col("STORE_PROVINCE"))
)

In [0]:
store_profiles_df = store_profiles_df.withColumn("LONGITUDE", lit(None).cast('double'))
store_profiles_df = store_profiles_df.withColumn("LATITUDE", lit(None).cast('double'))

windowPartition = Window.partitionBy(['BANNER','variable','YEAR'])
store_profiles_df = store_profiles_df.withColumn("BANNER_VALUE", sum('val').over(windowPartition)/sum('TOTAL_POPULATION').over(windowPartition))

windowPartition = Window.partitionBy(['Category','BANNER','STORE_NAME','YEAR'])
store_profiles_df = store_profiles_df.withColumn('BANNER_VALUE_PERCENT', col('BANNER_VALUE') / sum(when(col('LEAF_NODE_FLAG') == 1, col('BANNER_VALUE')).otherwise(0)).over(windowPartition)).select('STORE_CHAIN','BANNER',	'STORE_NAME',	'STORE_NUM',	'CHANNEL',	'SALES_GROUP',	'STORE_STREET','STORE_LONGITUDE','STORE_LATITUDE'	,'STORE_PROVINCE','Region',	'variable','Description',	'Language_Grouping',	'Category',	'YEAR',	'val',	'TOTAL_HOUSEHOLDS',	'TOTAL_HOUSEHOLD_POPULATION',	'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER',	'TOTAL_POPULATION',	'BANNER_VALUE_PERCENT','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG')

windowPartition = Window.partitionBy(['Region', 'CHANNEL','variable','YEAR'])
windowPartition2 = Window.partitionBy(['Region', 'CHANNEL','STORE_NUM','variable','YEAR'])
store_profiles_df = store_profiles_df.withColumn("CHANNEL_RE_VALUE", sum('val').over(windowPartition)/sum('TOTAL_POPULATION').over(windowPartition))


windowPartition = Window.partitionBy(['Region', 'CHANNEL','STORE_NAME','Category','YEAR'])
store_profiles_df = store_profiles_df.withColumn('CHANNEL_RE_VALUE_PERCENT', col('CHANNEL_RE_VALUE')/sum(when(col('LEAF_NODE_FLAG') == 1, col('CHANNEL_RE_VALUE')).otherwise(0)).over(windowPartition)).select('STORE_CHAIN',	'BANNER','STORE_NAME',	'STORE_NUM',	'CHANNEL',	'SALES_GROUP',	'STORE_STREET',	'STORE_LONGITUDE','STORE_LATITUDE','STORE_PROVINCE','Region','variable',	'Description',	'Language_Grouping',	'Category',	'YEAR',	'val',	'TOTAL_HOUSEHOLDS',	'TOTAL_HOUSEHOLD_POPULATION',	'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER',	'TOTAL_POPULATION',	'BANNER_VALUE_PERCENT','CHANNEL_RE_VALUE_PERCENT','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG')


windowPartition = Window.partitionBy(['Region','variable','YEAR'])
windowPartition2 = Window.partitionBy(['Region','STORE_NUM','variable','YEAR'])
store_profiles_df = store_profiles_df.withColumn("REGION_VALUE", sum('val').over(windowPartition)/sum('TOTAL_POPULATION').over(windowPartition))

windowPartition = Window.partitionBy(['Region','STORE_NAME','Category'])
store_profiles_df = store_profiles_df.withColumn('REGION_VALUE_PERCENT', col('REGION_VALUE')/sum(when(col('LEAF_NODE_FLAG') == 1, col('REGION_VALUE')).otherwise(0)).over(windowPartition)).select('STORE_CHAIN','BANNER',	'STORE_NAME',	'STORE_NUM',	'CHANNEL',	'SALES_GROUP',	'STORE_STREET',	'STORE_LONGITUDE','STORE_LATITUDE','STORE_PROVINCE','Region','variable',	'Description',	'Language_Grouping',	'Category',	'YEAR',	'val',	'TOTAL_HOUSEHOLDS',	'TOTAL_HOUSEHOLD_POPULATION',	'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER',	'TOTAL_POPULATION',	'BANNER_VALUE_PERCENT','CHANNEL_RE_VALUE_PERCENT','REGION_VALUE_PERCENT','Hierarchy_Description','MEDIAN_AVERAGE','ADD_TO_PROFILE','LEAF_NODE_FLAG','TOTAL_FLAG')


windowPartition = Window.partitionBy(['CHANNEL','variable','YEAR'])
windowPartition2 = Window.partitionBy(['STORE_NUM', 'CHANNEL','variable','YEAR'])#['STORE_NUM', 'CUSTOMER_GROUP'])
store_profiles_df = store_profiles_df.withColumn("CHANNEL_VALUE", sum('val').over(windowPartition)/sum('TOTAL_POPULATION').over(windowPartition))

windowPartition = Window.partitionBy(['CHANNEL','STORE_NAME','Category','YEAR'])
store_profiles_df = store_profiles_df.withColumn('CHANNEL_VALUE_PERCENT', col('CHANNEL_VALUE')/sum(when(col('LEAF_NODE_FLAG') == 1, col('CHANNEL_VALUE')).otherwise(0)).over(windowPartition)).select('STORE_CHAIN','BANNER',	'STORE_NAME',	'STORE_NUM',	'CHANNEL',	'SALES_GROUP',	'STORE_STREET',	'STORE_LONGITUDE','STORE_LATITUDE','STORE_PROVINCE','Region','variable',	'Description',	'Language_Grouping',	'Category',	'YEAR',	'val',	'TOTAL_HOUSEHOLDS',	'TOTAL_HOUSEHOLD_POPULATION',	'TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER',	'TOTAL_POPULATION',	'BANNER_VALUE_PERCENT','CHANNEL_RE_VALUE_PERCENT','REGION_VALUE_PERCENT','CHANNEL_VALUE_PERCENT','Hierarchy_Description','MEDIAN_AVERAGE','LEAF_NODE_FLAG','TOTAL_FLAG')
# display(store_profiles_df)


In [0]:
store_profiles_df.write.mode("overwrite").option("overwriteSchema", "True").format("delta").saveAsTable(
    "hyper.demostats_store_profiles_immi_combined_1015"
)

In [0]:
store_profiles_df = spark.sql("""
  SELECT * from  hyper.demostats_store_profiles_immi_combined_1015
""")


In [0]:
# windowPartition = Window.partitionBy(["BANNER","STORE_NUM","Category","YEAR"])

df_total = store_profiles_df.filter(col("TOTAL_FLAG") == 1).select("STORE_NAME","STORE_NUM","Category", "YEAR","val").withColumnRenamed("val", "TOTAL_CATEGORY")

df_with_total_2 = store_profiles_df.join(df_total, on=["Category","STORE_NUM","STORE_NAME","YEAR"], how="left")

# df_with_total_2=df_with_total_2.drop('STORE_VALUE')

df_with_total_2 = df_with_total_2.withColumn(
    'STORE_VALUE',
    when(col('Description') == 'Household Population For Recent Immigration By Place Of Birth', col('val')/col('TOTAL_HOUSEHOLD_POPULATION')).otherwise(col('val')/col('TOTAL_CATEGORY'))
)

df_with_total_2 = df_with_total_2.withColumn(
    'STORE_VALUE',
    when((col('LEAF_NODE_FLAG') == 0) & (col('TOTAL_FLAG') == 0), 0).otherwise(col('STORE_VALUE'))
)

df_with_total_2=df_with_total_2.drop('MEDIAN_AVERAGE','LEAF_NODE_FLAG','TOTAL_FLAG')


In [0]:

df_with_total_2 = df_with_total_2.select(
    "Category",
    "STORE_NUM",
    "STORE_NAME",
    "STORE_CHAIN",
    "BANNER",
    "CHANNEL",
    "SALES_GROUP",
    "STORE_STREET",
    "STORE_LONGITUDE",
    "STORE_LATITUDE",
    "STORE_PROVINCE",
    "Region",
    "variable",
    "Description",
    "Language_Grouping",
    "YEAR",
    "val",
    "TOTAL_HOUSEHOLDS",
    "TOTAL_HOUSEHOLD_POPULATION",
    "TOTAL_HOUSEHOLD_POPULATION_15_YEARS_OR_OVER",
    "TOTAL_POPULATION",
    "BANNER_VALUE_PERCENT",
    "CHANNEL_RE_VALUE_PERCENT",
    "REGION_VALUE_PERCENT",
    "CHANNEL_VALUE_PERCENT",
    "Hierarchy_Description",
    "TOTAL_CATEGORY",
    "STORE_VALUE",
)


display(df_with_total_2)

In [0]:
df_with_total_2.write.mode("overwrite").option("overwriteSchema", "True").format("delta").saveAsTable(
    "hyper.demostats_store_immi_combined_1015"
)

In [0]:
df.write.mode(
"overwrite"
).insertInto(
"hyper.demostats_store_profiles"
)

In [0]:
df_overwrite= spark.sql("""
                        select * from hyper.demostats_store_immi_combined_1015
                        """)

In [0]:
df_overwrite = df_overwrite.withColumn("REVENUE_CY", lit(0).cast("double")).withColumn("OUTLIER", lit('').cast("string"))

In [0]:
df_overwrite.write.mode(
"overwrite"
).insertInto(
"hyper.demostats_store_profiles"
)

In [0]:
df_overwrite= spark.sql("""
                        select * from hyper.master_demostats
                        """)

In [0]:

df_overwrite.write.mode(
"overwrite"
).insertInto(
"hyper.dim_master_demostats"
)